# Search for clear Planet Images covering a GeoJSON-specified area, download, and tile into 224x224 squares

## Created by Julian Lee, April 8 2021

In [1]:
## Import Packages - if not installed, use conda or pip to install

import requests
import os
import utm
import rasterio as rio
import shapely.geometry
import datetime


from pathlib import Path
from dateutil.relativedelta import relativedelta
from itertools import product
from rasterio import windows
from rasterio.mask import mask, raster_geometry_mask
from rasterio.io import MemoryFile

In [2]:
## Setup API Authentication

os.environ['PL_API_KEY'] = 'ef1d2935cae84b3cb4e4cadd95a0779e'
PLANET_API_KEY = os.getenv('PL_API_KEY')
url = 'https://api.planet.com/data/v1/'

session = requests.Session()
session.auth = (PLANET_API_KEY, '')

#### User-defined parameters

In [24]:
## Tile width & height
tile_width, tile_height = 224, 224

## Define overlap of tile windows in px
width_overlap, height_overlap = 20, 20

## Relative directory path to save tiles
out_path = './images/Planet/'+ datetime.date.today().isoformat() + '/'

## Latest day you look for should be how many days before today? Defaults to 0, starting from today.
start_days_back = 28

## Number of days before start day to search for clear imagery. Can increase if no results.
days_range = 88

## Minimum image clarity percentage, determined by Planet, to filter. Can lower if no results, but may give cloudy images. 
clear_threshold = 70

## GeoJSON Feature area of interest: replace with your own. GeoJSONs can be generated from drawings at https://www.geojson.io
geojson =    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -64.85023498535156,
              -9.386741925294897
            ],
            [
              -64.84268188476562,
              -9.403000374334917
            ],
            [
              -64.84474182128906,
              -9.421967599204851
            ],
            [
              -64.82139587402344,
              -9.468703809001507
            ],
            [
              -64.83787536621092,
              -9.528300181727518
            ],
            [
              -64.86328125,
              -9.535071839725905
            ],
            [
              -64.88250732421875,
              -9.539134770004505
            ],
            [
              -64.90242004394531,
              -9.570282292235254
            ],
            [
              -64.92713928222656,
              -9.579084335882534
            ],
            [
              -64.96353149414062,
              -9.579761406713759
            ],
            [
              -64.99031066894531,
              -9.554708887391607
            ],
            [
              -65.01228332519531,
              -9.557417356841295
            ],
            [
              -65.05210876464844,
              -9.586532040741064
            ],
            [
              -65.11184692382812,
              -9.597364774080114
            ],
            [
              -65.10498046875,
              -9.614967226987291
            ],
            [
              -65.04936218261719,
              -9.601426959820937
            ],
            [
              -65.01296997070312,
              -9.575698961474112
            ],
            [
              -64.99168395996094,
              -9.575021882542638
            ],
            [
              -64.96078491210938,
              -9.594656623196652
            ],
            [
              -64.94224548339844,
              -9.590594356301839
            ],
            [
              -64.91958618164062,
              -9.606843131687073
            ],
            [
              -64.88388061523438,
              -9.590594356301839
            ],
            [
              -64.88388061523438,
              -9.572990637650243
            ],
            [
              -64.87014770507812,
              -9.550646142809997
            ],
            [
              -64.82757568359375,
              -9.54522907463891
            ],
            [
              -64.80216979980469,
              -9.4714129598364
            ],
            [
              -64.82414245605469,
              -9.42264498083553
            ],
            [
              -64.8248291015625,
              -9.394871245232979
            ],
            [
              -64.85023498535156,
              -9.386741925294897
            ]
          ]
        ]
      }
    }

#### Setup search filters

In [25]:
item_types = ['PSScene3Band', 'PSScene4Band']
end = datetime.datetime.utcnow() # <-- get current day in UTC
start = datetime.datetime.utcnow() - relativedelta(days=days_range)  

fltr = {
    "type": "AndFilter",
    "config": [
        {
            "type": "DateRangeFilter", 
            "field_name":"acquired",
            "config": {
              "gt": start.isoformat("T") + "Z",
              "lte": end.isoformat("T") + "Z"
           }
        },
        {
           "type": "GeometryFilter",
           "field_name":"geometry",
           "config": geojson['geometry']
        },
        {
           "type":"RangeFilter",
           "field_name":"clear_percent",
           "config":{
              "gte": clear_threshold
            }
        },
        {
         "type":"PermissionFilter",
         "config": [
                "assets:download"
            ]
        }
    ]
}

#### Send search to Planet API using given filters and choose the result with best coverage of our AOI

In [26]:
res = session.post(url + 'quick-search', json = {'filter': fltr, 'item_types': item_types})
features = res.json()['features']
aoi = shapely.geometry.shape(geojson['geometry'])

best_feature = None
best_coverage = 0

print('number of matching features:', len(features))

for feature in features:
    intersection = shapely.geometry.shape(feature['geometry']).intersection(aoi).area 
    if intersection > best_coverage: 
        best_feature = feature
        best_coverage = intersection
    
print('Best Coverage:', best_coverage * 100 / aoi.area, '%, Clear Percentage:', best_feature['properties']['clear_percent'], '%')

number of matching features: 55
Best Coverage: 80.44702747928407 %, Clear Percentage: 88 %


#### Send activation request to Planet API for the best result - this prepares the image for download

In [27]:
order_url = 'https://api.planet.com/compute/ops/orders/v2'

json = {
    'name': 'order-' + datetime.datetime.utcnow().isoformat(), 
    'subscription_id': 544465,
    'products': [
        {
            'item_ids': [best_feature['id']],
            'item_type': best_feature['properties']['item_type'],
            'product_bundle': 'analytic'
        }
    ],
    'delivery': {
        'single_archive': True
    },
    'notifications': {
        'email': True
    },
    'order_type': 'partial',
    'tools': [
        {
            'clip': {
                'aoi': geojson['geometry']   
            }
        }
    ]
}

In [28]:
if best_feature:
    res = session.post(order_url, json=json)
    if res.status_code == 202:
    ## This should output 202 
        print('Search successful, activation request status: ', res.status_code)
    else:
        print('Search unsuccessful, error: ', res.text)
else:
    print('No search results, widen filters')

Search successful, activation request status:  202
